In [1]:
from cartoframes.__version__ import __version__
print(__version__)
# note: this is actually from `develop`: pip install -e git+https://github.com/cartodb/cartoframes.git@develop#egg=cartoframes

1.0b2


In [4]:
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.client import SQLClient
from cartoframes.data import Dataset
from cartoframes.viz import Map, Layer
import pandas as pd
import geopandas as gpd

creds_here = Credentials('here-testing-admin', '456dc13794b04c79c34aff64bcfd08cea7a1749b')
# creds_andy = Credentials('eschbacher', '01c1be0f2edf4707024f448eaff513552a0b0b4b')

set_default_credentials(creds_here)

station_points = 'capital_bikeshare_stations_points_arlington'
usage_data = 'capitalbikeshare_tripdata_201907'

sql_client = SQLClient(creds_here)

In [5]:
Dataset(usage_data).download(limit=2)

,the_geom,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type
cartodb_id,,,,,,,,,,
1,,443.0,2019-07-01 04:11:52,2019-07-01 04:19:16,31603.0,1st & M St NE,31620.0,5th & F St NW,W24210,Member
2,,500.0,2019-07-01 06:22:18,2019-07-01 06:30:39,31244.0,4th & E St SW,31652.0,4th & M St SE,W24154,Member


In [6]:
Dataset(station_points).download(limit=2)

,the_geom,objectid,gisid,name
cartodb_id,,,,
1,0101000020E610000085093973704353C036DDEA41B96D...,37.0,31001.0,18th & Eads St
2,0101000020E6100000712065E48A4453C078CEF79FE072...,1.0,31014.0,Lynn & 19th St N


In [426]:
from cartoframes.viz.helpers import size_continuous_layer, color_continuous_layer, size_bins_layer, size_category_layer, color_bins_layer, color_category_layer

### Visualize the data

In [266]:
sql = f'''
SELECT
    _ends.num_bike_dropoffs,
    _starts.num_bike_pickups,
    abs(_ends.num_bike_dropoffs - _starts.num_bike_pickups) as diff,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 1
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 0
         ELSE -1 END as diff_sign,
    _ends.num_bike_dropoffs - _starts.num_bike_pickups as diff_relative,
    _starts.station_id,
    row_number() OVER () as cartodb_id,
    ST_X(_starts.the_geom) as longitude,
    ST_Y(_starts.the_geom) as latitude,
    _starts.the_geom,
    ST_Transform(_starts.the_geom, 3857) as the_geom_webmercator
FROM
    (SELECT
      count(u.*) as num_bike_dropoffs,
      u.end_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id
    FROM {station_points} as s
    JOIN {usage_data} as u
    ON u.end_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4) as _ends
JOIN
    (SELECT
      count(u.*) as num_bike_pickups,
      u.start_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id
    FROM {usage_data} as u
    JOIN {station_points} as s
    ON u.start_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4) as _starts
ON _ends.station_id = _starts.station_id
'''
Map([
    size_continuous_layer(Dataset(sql), 'num_bike_pickups'),
#     color_continuous_layer(Dataset(sql), 'diff_relative')
    size_continuous_layer(Dataset(sql), 'num_bike_dropoffs', color='purple')  
])

### Create a table from the query above

In [267]:
#creates a table from sql query
pickup_dropoff_points = 'capital_bikeshare_july_2019_pickup_dropoff'
Dataset(sql).upload(table_name=pickup_dropoff_points, if_exists='replace')

/Users/mamataakella/.local/share/virtualenvs/jupyter-cartoframes-bAYrQlfl/src/carto/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: f6628627-5c66-4896-ad00-9180a4fb197b
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


In [268]:
# absolute value 
from cartoframes.viz import basemaps
Map([
#     size_continuous_layer(pickup_dropoff_points, 'num_bike_pickups'),
#     color_continuous_layer(Dataset(sql), 'diff_relative')
    size_continuous_layer(pickup_dropoff_points, 'diff', color='yellow * $diff_sign')  
],
    basemap=basemaps.voyager)

In [396]:
sql = f'''
SELECT
    _ends.num_bike_dropoffs,
    _starts.num_bike_pickups,
    abs(_ends.num_bike_dropoffs - _starts.num_bike_pickups) as diff,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 'Drop off'
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 'Equal'
         ELSE 'Pick up' END as diff_usage,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 1
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 0
         ELSE -1 END as diff_sign,
    _ends.num_bike_dropoffs - _starts.num_bike_pickups as diff_relative,
    _starts.station_id,
    row_number() OVER () as cartodb_id,
    ST_X(_starts.the_geom) as longitude,
    ST_Y(_starts.the_geom) as latitude,
    _starts.the_geom,
    ST_Transform(_starts.the_geom, 3857) as the_geom_webmercator
FROM
    (SELECT
      count(u.*) as num_bike_dropoffs,
      u.end_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id
    FROM {station_points} as s
    JOIN {usage_data} as u
    ON u.end_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4) as _ends
JOIN
    (SELECT
      count(u.*) as num_bike_pickups,
      u.start_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id
    FROM {usage_data} as u
    JOIN {station_points} as s
    ON u.start_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4) as _starts
ON _ends.station_id = _starts.station_id
'''
color_bins_layer(sql,'diff_sign')

In [270]:
# total for each station size and color indicates either more pu or do
from cartoframes.viz import WidgetList
value = 'diff'
title = 'Drop Offs '
diff_map = Layer(
        sql,
        style={
            'point': {
                'width': 10,
                'color': 'ramp($diff_usage,bold)',
            }
        },
        popup={
            'hover': [{
                'title': title,
                'value': f'$diff_usage'
            }]
        },
        legend={
            'type': {
                'point': 'color-category-point',
                'line': 'size-continuous-line',
                'polygon': 'size-continuous-polygon'
            },
            'title': title,
            'description': '',
            'footer': ''
        }
    )
diff_map

### Animating pickups/dropoffs over time

In [397]:
sql = f'''
SELECT
    _ends.num_bike_dropoffs,
    _starts.num_bike_pickups,
    abs(_ends.num_bike_dropoffs - _starts.num_bike_pickups) as diff,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 1
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 0
         ELSE -1 END as diff_sign,
    _ends.num_bike_dropoffs - _starts.num_bike_pickups as diff_relative,
    _starts.station_id,
    row_number() OVER () as cartodb_id,
    ST_X(_starts.the_geom) as longitude,
    ST_Y(_starts.the_geom) as latitude,
    _starts.the_geom,
    ST_Transform(_starts.the_geom, 3857) as the_geom_webmercator,
    _ends.day_of_month::numeric as day_of_month
FROM
    (SELECT
      count(u.*) as num_bike_dropoffs,
      u.end_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id,
      EXTRACT(DAY FROM end_date) as day_of_month
    FROM {station_points} as s
    JOIN {usage_data} as u
    ON u.end_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4, 5) as _ends
JOIN
    (SELECT
      count(u.*) as num_bike_pickups,
      u.start_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id,
      EXTRACT(DAY FROM start_date) as day_of_month
    FROM {usage_data} as u
    JOIN {station_points} as s
    ON u.start_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4, 5) as _starts
ON _ends.station_id = _starts.station_id and _ends.day_of_month = _starts.day_of_month
'''

size_continuous_layer(Dataset(sql), 'diff', animate='day_of_month', widget=True)

In [399]:
sql = f'''
SELECT
    _ends.num_bike_dropoffs,
    _starts.num_bike_pickups,
    abs(_ends.num_bike_dropoffs - _starts.num_bike_pickups) as diff,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 'Drop off'
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 'Equal'
         ELSE 'Pick up' END as diff_usage,
    CASE WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups > 0 THEN 1
         WHEN _ends.num_bike_dropoffs - _starts.num_bike_pickups = 0 THEN 0
         ELSE -1 END as diff_sign,
    _ends.num_bike_dropoffs - _starts.num_bike_pickups as diff_relative,
    _starts.station_id,
    row_number() OVER () as cartodb_id,
    ST_X(_starts.the_geom) as longitude,
    ST_Y(_starts.the_geom) as latitude,
    _starts.the_geom,
    ST_Transform(_starts.the_geom, 3857) as the_geom_webmercator,
    _ends.day_of_month::numeric as day_of_month,
    make_date(2019, 07, _ends.day_of_month::int) as date_of_rides
FROM
    (SELECT
      count(u.*) as num_bike_dropoffs,
      u.end_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id,
      EXTRACT(DAY FROM end_date) as day_of_month
    FROM {station_points} as s
    JOIN {usage_data} as u
    ON u.end_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4, 5) as _ends
JOIN
    (SELECT
      count(u.*) as num_bike_pickups,
      u.start_station_number::int as station_id,
      s.the_geom,
      s.cartodb_id,
      EXTRACT(DAY FROM start_date) as day_of_month
    FROM {usage_data} as u
    JOIN {station_points} as s
    ON u.start_station_number::int = s.gisid::int
    GROUP BY 2, 3, 4, 5) as _starts
ON _ends.station_id = _starts.station_id and _ends.day_of_month = _starts.day_of_month
'''

size_continuous_layer(Dataset(sql), 'diff', animate='date_of_rides')

In [400]:
pickup_dropoff_points_v3 = 'pick_up_dropoff_points_v3'
Dataset(sql).upload(table_name=pickup_dropoff_points_v3,if_exists="replace")

/Users/mamataakella/.local/share/virtualenvs/jupyter-cartoframes-bAYrQlfl/src/carto/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 4a768ae0-4501-45c6-8d49-c329a6e2a012
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


In [263]:
# print table name
print(pickup_dropoff_points)

capital_bikeshare_july_2019_pickup_dropoff


In [364]:
cols = set(Dataset(pickup_dropoff_points).download(limit=1).columns) - {'the_geom', 'cartodb_id', 'the_geom_webmercator'}
sql = f'''
-- CREATE TABLE {pickup_dropoff_points}_isochrone_300 AS
SELECT ST_MakeValid((cdb_isochrone(the_geom, 'walk', Array[300])).the_geom) as the_geom, cartodb_id,
    {','.join(cols)}
FROM {pickup_dropoff_points};
SELECT CDB_Cartodbfytable('here-testing-admin','{pickup_dropoff_points}_isochrone_300');
'''

sql_client.execute(sql)

/Users/mamataakella/.local/share/virtualenvs/jupyter-cartoframes-bAYrQlfl/src/carto/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 57dd4c1e-2f5b-4cb5-bafa-653294957235
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


{'user': 'here-testing-admin',
 'status': 'done',
 'query': "-- CREATE TABLE capital_bikeshare_july_2019_pickup_dropoff_isochrone_300 AS\nSELECT ST_MakeValid((cdb_isochrone(the_geom, 'walk', Array[300])).the_geom) as the_geom, cartodb_id,\n    diff,diff_relative,latitude,diff_sign,num_bike_pickups,num_bike_dropoffs,station_id,longitude\nFROM capital_bikeshare_july_2019_pickup_dropoff;\nSELECT CDB_Cartodbfytable('here-testing-admin','capital_bikeshare_july_2019_pickup_dropoff_isochrone_300');",
 'created_at': '2019-08-23T17:08:25.489Z',
 'updated_at': '2019-08-23T17:08:41.218Z',
 'job_id': '57dd4c1e-2f5b-4cb5-bafa-653294957235'}

In [379]:
Layer('capital_bikeshare_july_2019_pickup_dropoff_isochrone_300')

In [366]:
from cartoframes.client import DataObsClient
do = DataObsClient(creds_here)

In [ ]:
## TODO: https://cartodb.github.io/bigmetadata/united_states/transportation.html

In [367]:
# ratio of households with no car will be converted to percent in later step
do_aug = do.augment(
    f'{pickup_dropoff_points}_isochrone_300',
    [{"numer_id": "us.census.acs.B08201002",
    "denom_id": "us.census.acs.B11001001",
    "normalization": "denominated"}]
)

In [368]:
do_aug.dataframe['no_cars_perc'] = [round(r * 100.0, 2) for r in do_aug.dataframe['no_cars_2011_2015_by_households']]

In [369]:
do_aug.dataframe.head()

,diff,diff_relative,the_geom,latitude,diff_sign,num_bike_pickups,num_bike_dropoffs,station_id,longitude,no_cars_2006_2010_by_households,no_cars_2010_2014_by_households,no_cars_2011_2015_by_households,no_cars_perc
cartodb_id,,,,,,,,,,,,,
1,26.0,-26.0,0106000020E61000000100000001030000000100000019...,38.858726,-1.0,204.0,178.0,31000.0,-77.053144,0.163147,0.151932,0.193601,19.36
2,54.0,-54.0,0106000020E61000000100000001030000000100000025...,38.857216,-1.0,276.0,222.0,31001.0,-77.053738,0.164142,0.160701,0.200308,20.03
3,129.0,129.0,0106000020E6100000010000000103000000010000001F...,38.856372,1.0,710.0,839.0,31002.0,-77.049218,0.126857,0.166137,0.160758,16.08
4,2.0,-2.0,0106000020E6100000010000000103000000010000001C...,38.860167,-1.0,489.0,487.0,31003.0,-77.049614,0.138377,0.157942,0.169418,16.94
5,3.0,3.0,0106000020E61000000100000001030000000100000014...,38.857937,1.0,227.0,230.0,31004.0,-77.059552,0.218141,0.160877,0.140935,14.09


In [375]:
# upload augmented data to account, say ok to overwrite previous version
do_aug.upload(table_name=f'{pickup_dropoff_points}_isochrone_augmented_300',if_exists="replace")

/Users/mamataakella/.local/share/virtualenvs/jupyter-cartoframes-bAYrQlfl/src/carto/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 32d816d7-284f-4f6a-be42-e28c121364b5
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


In [378]:
Dataset(f'{pickup_dropoff_points}_isochrone_augmented_300').download()

ValueError: We can not detect the Dataset type

In [458]:
# diff_sign whether pickups are larger than dropoffs -1: more pu than do // 0: equal // 1: do>pu
from cartoframes.viz.widgets import time_series_widget, animation_widget

Map([
    color_bins_layer(
        'capital_bikeshare_july_2019_pickup_dropoff_isochrone_augmented_', 
        'no_cars_perc', 
        '%Carfree Households', 
        bins=7, 
        popup=False
    ),
    Layer(
        'pick_up_dropoff_points_v3',
        '''
        width: ramp($diff,[5,50])*animation($date_of_rides,20,fade(0.5,0.5))
        strokeColor: ramp($diff_usage,[rgb(1,214,254),rgb(254,1,197),white])
        color: opacity(ramp($diff_usage,[rgb(1,214,254),rgb(254,1,197),white]),0.1)
        ''',
        legend={
            'type': 'color-category-point',
            'prop': 'strokeColor'
        },
        #widgets=[animation_widget(prop='width')]
    ),
    
    Layer('pick_up_dropoff_day','width:2 color: darkorange'),

],
basemap=basemaps.darkmatter,
show_info=True)

### Run a Model

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipeline.fit(do_aug.dataframe[['num_bike_pickups', 'num_bike_dropoffs', 'pop_25_64_2011_2015_by_total_pop']])

### Add results to dataset

In [ ]:
do_aug.dataframe['labels'] = pipeline.steps[1][1].labels_.astype(str)

In [ ]:
do_aug.dataframe.head()

In [ ]:
do_aug.upload(table_name='demo_augmentation', if_exists='replace')

In [45]:
results_map = Map(
    color_continuous_layer(f'{pickup_dropoff_points}_isochrone_augmented', 'no_cars_perc', widget=True, palette='prism'),
    Layer('pick_up_dropoff_day')
    
)
results_map

In [ ]:
results_map.publish('Station Encoding')

### Map Grid

In [ ]:
from cartoframes.viz import MapGrid
results_map = Map(
    color_category_layer('demo_augmentation', 'labels')
)
MapGrid([
    results_map,
    Map(diff_map)
], 2, 1, viewport={'zoom': 11})